<a href="https://colab.research.google.com/github/venkat23-hub/Brain_tumor_identification/blob/main/Brain_tumor_identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opencv-python


Import libraries

In [9]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


Load & Preprocess Images (COLAB PATH FIXED)

In [3]:
data = []
labels = []

IMG_SIZE = 128
dataset_path = "/content/drive/MyDrive/Brain_tumor_identification/brain_tumor_dataset"

for folder in ["yes", "no"]:
    folder_path = os.path.join(dataset_path, folder)
    label = 1 if folder == "yes" else 0

    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)

        img = cv2.imread(img_path)
        if img is None:
            continue

        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        data.append(img)
        labels.append(label)

print("Total images loaded:", len(data))


Total images loaded: 253


Convert to NumPy Arrays

In [4]:
data = np.array(data).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
labels = np.array(labels)

data = data / 255.0
labels = to_categorical(labels, 2)


Train-Test Split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.2, random_state=42
)


Build CNN Model

In [6]:
model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     7,372,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,392,002 (28.20 MB)

 Trainable params: 7,392,002 (28.20 MB)

 Non-trainable params: 0 (0.00 B)

Compile Model

In [7]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


Train Model

In [8]:
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=16,
    validation_split=0.1
)


Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 505ms/step - accuracy: 0.5956 - loss: 0.7490 - val_accuracy: 0.7619 - val_loss: 0.4830
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 326ms/step - accuracy: 0.7974 - loss: 0.4950 - val_accuracy: 0.8571 - val_loss: 0.4502
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 385ms/step - accuracy: 0.8384 - loss: 0.4000 - val_accuracy: 0.8095 - val_loss: 0.5351
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 502ms/step - accuracy: 0.8682 - loss: 0.3279 - val_accuracy: 0.9048 - val_loss: 0.5434
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 353ms/step - accuracy: 0.8960 - loss: 0.2131 - val_accuracy: 0.9048 - val_loss: 0.5564
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 442ms/step - accuracy: 0.9505 - loss: 0.1347 - val_accuracy: 0.8571 - val_loss: 0.6948
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 344ms/step - accuracy: 0.9757 - loss: 0.0827 - val_accuracy: 0.9048 - val_loss: 0.7599
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 375ms/step - accuracy: 0.9696 - loss: 0.0852 - val_accuracy: 0.

Evaluate model

In [10]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy * 100)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - accuracy: 0.8564 - loss: 0.4400
Test Accuracy: 86.27451062202454


Get Model Predictions

In [11]:
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = np.argmax(y_test, axis=1)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 297ms/step


Calculate Evaluation Metrics

In [12]:
accuracy  = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall    = recall_score(y_true, y_pred)
f1        = f1_score(y_true, y_pred)

print("Accuracy  :", accuracy)
print("Precision :", precision)
print("Recall    :", recall)
print("F1-score  :", f1)


Accuracy  : 0.8627450980392157
Precision : 0.8333333333333334
Recall    : 0.9259259259259259
F1-score  : 0.8771929824561403


Detailed Classification Report

In [13]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred, target_names=["Normal", "Tumor"]))


              precision    recall  f1-score   support

      Normal       0.90      0.79      0.84        24
       Tumor       0.83      0.93      0.88        27

    accuracy                           0.86        51
   macro avg       0.87      0.86      0.86        51
weighted avg       0.87      0.86      0.86        51

